# Titanic_02 : 타이타닉 
---
- 타이타닉에 탑승한 사람들의 신상정보를 활용
- 승선한 사람들의 생존여부를 예측하는 모델을 생성
    - 여러 시각화 도구(matplotlib, seaborn, plotly), 데이터 분석 도구(pandas, numpy), 머신 러닝 도구(sklearn)을 사용


## 가설
---
1. Pclass :  좋은 객실을 사용한 승객이 더 많이 살아남았을 것이라는 가설 
2. Age : 젊은사람일수록 에너지가 있으니까 더 많이 살아 남았을거라는 가설 
3. Fare : 탑승료에 따른 생존률, 근데 이거는 pclass와 겹치거나 할것같음
4. sex : 여성이 더 많이 살아 남았을것이라는 가설 

In [1]:
# import
import numpy as np    # 넘파이
import pandas as pd   # 판다스 
import pandas as Series # 판다스 시리즈 임포트 

#  그래프 
import matplotlib.pyplot as plt   # 맷폴렛- 그래프
%matplotlib inline
import seaborn as sns      # 시본 - 그래프 
import missingno as msno   # missingno =>  missingno 패키지를 사용하면 누락데이터에 대한 시각화

plt.style.use('seaborn')
sns.set(font_scale=2.5)
# matplotlib 의 기본 scheme 말고 seaborn scheme 을 세팅하고, 
# 일일이 graph 의 font size 를 지정할 필요 없이 seaborn 의 font_scale 을 사용하면 편합니다.


# ?? 
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go 
import plotly.tools as tls 


# ignore warnings
import warnings    # 워닝 방지
warnings.filterwarnings('ignore')



In [2]:
# ! pip install plotly

In [3]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

In [4]:
# Titanic_01에서 진행한 내용 추가 

In [5]:
# FamilySize -> 학습, 테스트 모두 적용
# 자기자신을 포함해야하기 때문에 1을 더한다
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1 
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1 

In [6]:
df_test.loc[df_test.Fare.isnull(),'Fare'] = df_test['Fare'].mean()

In [7]:
df_train['Fare'] = df_train['Fare'].map(lambda i :np.log(i) if i > 0 else 0)
df_test['Fare'] = df_test['Fare'].map(lambda i :np.log(i) if i > 0 else 0)

# 3.Feature engineering
--- 

- 본격적인 Feature engineering 시작 
- 가장 먼저 데이터셋에 존재하는 결측값을 채울것이다.     

- 의미없는 숫자로 채울 수는 없고, 결측데이터를 포함하는 feature의 statistics(통계)를 참고하거나, 다른 아이디어를 생각해내서 채울 수 있다.     
- 결측 데이터를 어떻게 채우는지에 따라 모델의 성능이 좌지우지될 수 있기 때문에, 신경써줘야할 부분이다.

- Feature engineering은 실제 모델의 학습에 쓰려고 하는것 
- 학습, 테스트 모두 똑같이 적용해줘야한다!!!!!

## 결측치 채우기(Fill Null)
---

### 네임타이틀과 나이를 이용해 결측치 채우기 : Fill Null in Age using title

- 나이에는 177개의 결측치가 있다.
- 결측치를 채울 수 있는 여러가지 아이디어중 title + statistics를 사용 
- 영어에서는 Miss, Mrr, Mrs 같은 title이 존재
    - 각 승객의 이름에는 꼭 타이틀네임이 붙는다.
    - 이를 이용해서 진행 
    
- 판다스의 시리즈를 이용 
    - data -> string으로 바꾸어주는  str method가 존재 
    - 추가로 정규 표현식을 적용하게 해주는 extract method 존재 
    - 위 메소드로 쉽게 타이틀을 추출 가능 -> 타이틀을 Initial column에 저장
    

In [8]:
# 문자열 추가 
df_train.Name

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
886                                Montvila, Rev. Juozas
887                         Graham, Miss. Margaret Edith
888             Johnston, Miss. Catherine Helen "Carrie"
889                                Behr, Mr. Karl Howell
890                                  Dooley, Mr. Patrick
Name: Name, Length: 891, dtype: object

In [9]:
# 이름 컬럼안에 있는 네임 타이틀을 가져온다.
# . 앞에 있는 글자를 가져온다 
df_train['Initial'] = df_train.Name.str.extract('([A-Za-z]+)\.')
df_test['Initial'] = df_test.Name.str.extract('([A-Za-z]+)\.')

In [10]:
df_train.Initial.value_counts()

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Major         2
Mlle          2
Col           2
Capt          1
Don           1
Countess      1
Sir           1
Ms            1
Jonkheer      1
Lady          1
Mme           1
Name: Initial, dtype: int64

In [11]:
df_test.Initial.value_counts()

Mr        240
Miss       78
Mrs        72
Master     21
Col         2
Rev         2
Dona        1
Ms          1
Dr          1
Name: Initial, dtype: int64

In [12]:
# pandas 의 crosstab 을 이용하여 우리가 추출한 Initial 과 Sex 간의 count확인 
# train
pd.crosstab(df_train['Initial'], df_train['Sex']).T.style.background_gradient(cmap = 'summer_r')

Initial,Capt,Col,Countess,Don,Dr,Jonkheer,Lady,Major,Master,Miss,Mlle,Mme,Mr,Mrs,Ms,Rev,Sir
Sex,,,,,,,,,,,,,,,,,
female,0,0,1,0,1,0,1,0,0,182,2,1,0,125,1,0,0
male,1,2,0,1,6,1,0,2,40,0,0,0,517,0,0,6,1


In [13]:
# pandas 의 crosstab 을 이용하여 우리가 추출한 Initial 과 Sex 간의 count확인 
# test
pd.crosstab(df_test['Initial'], df_test['Sex']).T.style.background_gradient(cmap = 'summer_r')

Initial,Col,Dona,Dr,Master,Miss,Mr,Mrs,Ms,Rev
Sex,,,,,,,,,
female,0,1,0,0,78,0,72,1,0
male,2,0,1,21,0,240,0,0,2


In [14]:
# 이름 표준화 
old_title = ['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona']
new_title = ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr']

print('old_title = ',old_title,'\n','new_title =',new_title)

old_title =  ['Mlle', 'Mme', 'Ms', 'Dr', 'Major', 'Lady', 'Countess', 'Jonkheer', 'Col', 'Rev', 'Capt', 'Sir', 'Don', 'Dona'] 
 new_title = ['Miss', 'Miss', 'Miss', 'Mr', 'Mr', 'Mrs', 'Mrs', 'Other', 'Other', 'Other', 'Mr', 'Mr', 'Mr', 'Mr']


In [15]:
# 크로스테이블을 참고해서 남자, 여자가 쓰는 initial을 구분 
# replace를 사용해서 특정 데이터 값을 원하는 값으로 치환 

df_train['Initial'].replace(old_title,new_title,inplace=True)
df_test['Initial'].replace(old_title,new_title,inplace=True)

In [16]:
# 네이밍변경이 잘도
df_train.Initial.value_counts()

Mr        529
Miss      186
Mrs       127
Master     40
Other       9
Name: Initial, dtype: int64

In [17]:
df_test.Initial.value_counts()

Mr        242
Miss       79
Mrs        72
Master     21
Other       4
Name: Initial, dtype: int64